In [1]:
import pandas as pd
import numpy as np
import sqlite3
import pickle
import gc
from IPython.display import clear_output

from functools import lru_cache

def sum_2_tuples(a, b):
    return tuple(map(sum, zip(a, b)))

File paths:

In [2]:
LOB_DIR = '../Data_lob/'

RANDOM_MOMENTS_FILE = '../0_CommonFiles/random_moments_aligned_to_1000_2000.pkl'
TWAP_FILE = './twap.db3'

Experiment details:

In [3]:
# TWAP hedger
volume = 1_000 # in contracts
share_1_hedge = 0.1

In [4]:
with open(RANDOM_MOMENTS_FILE, 'rb') as file:
    moments = pickle.load(file)

In [5]:
len(moments)

2712

# Functions

In [6]:
class LOBCache:
    def __init__(self):
        pass
    
    def get_lob(self, datetime):
        print(datetime)
        filename = datetime.strftime('LOB_%m%d.feather')
        return self.__get_lob_by_filename(filename)
        pass
    
    @lru_cache(maxsize=4)
    def __get_lob_by_filename(self, filename):
        print(filename)
        lob = pd.read_feather(LOB_DIR + filename).dropna()
        lob['BID_SIZE10'] = 99_999_999
        lob['ASK_SIZE10'] = 99_999_999
        return lob

In [7]:
def make_hedge(lob, datetime, size, side):
    #lob = lob[lob['Time'] >= datetime]
    index = lob.Time.searchsorted(datetime, side = 'right')
    lob = lob.iloc[index:]
    if len(lob) == 0:
        return None, lob
    row = lob.iloc[0]
    
    quote_side = 'BID' if (side == 'S') else 'ASK'
    
    maxsize = sum([row[quote_side + '_SIZE' + str(level)] for level in range(1, 11)])
        
    if maxsize < size:
        raise ValueError(f'size {size} is greater than available {maxsize}')
    
    size_hedged, sum_hedged = 0, 0.0
    level = 1
    while size_hedged < size:
        portion = min(size - size_hedged, row[quote_side + '_SIZE' + str(level)])
        price = row[quote_side + '_PRICE' + str(level)]
        size_hedged += portion
        sum_hedged += portion * price
        print(f'added {portion} by {price}')
        level += 1
        
    print('HEDGE ended')
    return (size_hedged, sum_hedged), lob

In [8]:
cache = LOBCache()

In [9]:
def calc_hedge_twap(cache, moment, interval_sec = 5):
    lob = cache.get_lob(moment)
    reduced_lob = lob
    remain = volume
    total = None
    
    if len(reduced_lob) == 0:
        return None
    
    for seconds in range(0, 100, interval_sec):
        time_param = moment + np.timedelta64(seconds, 's')
        print(time_param)

        size_contracts = min(int(volume * share_1_hedge), remain)
        hedge_result, reduced_lob = make_hedge(reduced_lob, time_param, size = size_contracts, side = side)
        while hedge_result is None and len(reduced_lob) > 0:
            print('bad orderbook', reduced_lob.iloc[0])
            reduced_lob = reduced_lob.iloc[1: , :]
            hedge_result, reduced_lob = make_hedge(reduced_lob, time_param, size = size_contracts, side = side)

        if hedge_result is None:
            break
            
        if total is None:
            total = hedge_result
        else:
            total = sum_2_tuples(total, hedge_result)
        remain -= hedge_result[0]
        if remain == 0:
            break
            
    #del lob
            
    if remain > 0:
        return None
    else:
        return total

# Calculation

In [10]:
#CREATE TABLE RESULT(MOMENT TEXT, SIDE TEXT, QUANTITY INTEGER, SUM REAL, PRIMARY KEY(MOMENT,SIDE));
twap_sql = sqlite3.connect(TWAP_FILE)
twap_sql.execute('CREATE TABLE IF NOT EXISTS RESULT(MOMENT TEXT, SIDE TEXT, QUANTITY INTEGER, SUM REAL, PRIMARY KEY(MOMENT,SIDE));')
cur=twap_sql.cursor()

In [11]:
records = []

for moment in moments:
    for side in ('B', 'S'):
        record = (str(moment),side)
        query='select exists(select 1 from result where moment=? and side=? collate nocase) limit 1'
        # 'query' RETURNS 1 IF USERNAME EXISTS OR 0 IF NOT, AS INTEGER(MAYBE). 'collate nocase'= CASE INSENSITIVE, IT'S OPTIONAL
        check=cur.execute(query,record) 
        if check.fetchone()[0]==1:
            print(f'Moment already available ' + str(record))
            continue
        result = calc_hedge_twap(cache, moment)

        query='INSERT OR REPLACE INTO result(moment, side, quantity, sum) VALUES(?, ?, ?, ?);'
        if result:
            record = (str(moment), side, int(result[0]), result[1])
        else:
            record = (str(moment), side, -1, -1)

        print(record)
        cur.execute(query, record)
        records.append(record)

        gc.collect()

        if len(records) % 100 == 0:
            clear_output(wait=False)

2021-10-06 13:09:37.144599
2021-10-06 13:09:37.144599
added 100 by 72.6975
HEDGE ended
2021-10-06 13:09:42.144599
added 100 by 72.6975
HEDGE ended
2021-10-06 13:09:47.144599
added 100 by 72.6975
HEDGE ended
2021-10-06 13:09:52.144599
added 100 by 72.6975
HEDGE ended
2021-10-06 13:09:57.144599
added 100 by 72.6975
HEDGE ended
2021-10-06 13:10:02.144599
added 100 by 72.6975
HEDGE ended
2021-10-06 13:10:07.144599
added 100 by 72.6975
HEDGE ended
2021-10-06 13:10:12.144599
added 100 by 72.6975
HEDGE ended
2021-10-06 13:10:17.144599
added 100 by 72.695
HEDGE ended
2021-10-06 13:10:22.144599
added 100 by 72.6975
HEDGE ended
('2021-10-06 13:09:37.144599', 'B', 1000, 72697.25000000001)
2021-10-06 13:09:37.144599
2021-10-06 13:09:37.144599
added 50 by 72.69
added 50 by 72.6875
HEDGE ended
2021-10-06 13:09:42.144599
added 100 by 72.6925
HEDGE ended
2021-10-06 13:09:47.144599
added 100 by 72.6925
HEDGE ended
2021-10-06 13:09:52.144599
added 100 by 72.6925
HEDGE ended
2021-10-06 13:09:57.144599
ad

2021-10-06 16:11:45.281590
2021-10-06 16:11:45.281590
added 100 by 72.395
HEDGE ended
2021-10-06 16:11:50.281590
added 100 by 72.4
HEDGE ended
2021-10-06 16:11:55.281590
added 100 by 72.4025
HEDGE ended
2021-10-06 16:12:00.281590
added 100 by 72.4
HEDGE ended
2021-10-06 16:12:05.281590
added 100 by 72.3975
HEDGE ended
2021-10-06 16:12:10.281590
added 100 by 72.3975
HEDGE ended
2021-10-06 16:12:15.281590
added 100 by 72.4
HEDGE ended
2021-10-06 16:12:20.281590
added 50 by 72.4025
added 50 by 72.4
HEDGE ended
2021-10-06 16:12:25.281590
added 100 by 72.4025
HEDGE ended
2021-10-06 16:12:30.281590
added 2 by 72.4025
added 98 by 72.4
HEDGE ended
('2021-10-06 16:11:45.281590', 'S', 1000, 72399.63)
2021-10-06 16:22:34.587892
2021-10-06 16:22:34.587892
added 100 by 72.4075
HEDGE ended
2021-10-06 16:22:39.587892
added 100 by 72.4075
HEDGE ended
2021-10-06 16:22:44.587892
added 100 by 72.4075
HEDGE ended
2021-10-06 16:22:49.587892
added 100 by 72.4075
HEDGE ended
2021-10-06 16:22:54.587892
added 

In [12]:
twap_sql.commit()

In [13]:
cur.close()
twap_sql.close()